<a href="https://colab.research.google.com/github/ac-26/Automated-Content-Tagging-Provision/blob/main/content_tagging_v6.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [10]:
import numpy as np
import pandas as pd
from sklearn.metrics.pairwise import cosine_similarity
from transformers import pipeline, AutoTokenizer, AutoModel
import torch
import re
from typing import List, Dict, Tuple
from collections import Counter
import spacy

In [11]:
class TextEncoder:
  #initialization function
  def __init__(self, model_name="sentence-transformers/all-MiniLM-L6-v2"):
      self.tokenizer = AutoTokenizer.from_pretrained(model_name)
      self.model = AutoModel.from_pretrained(model_name)
      self.model.eval()

  #encodes text
  def encode_text(self, text: str) -> np.ndarray:
    inputs = self.tokenizer(text, return_tensors="pt",truncation=True, padding=True, max_length=512)
    with torch.no_grad():
      outputs = self.model(**inputs)
    embeddings = outputs.last_hidden_state.mean(dim=1)
    return embeddings.numpy().flatten()

### **This was my initial approach when I was brainstorming to build this system, this is not being used currently in our system, IT IS ONLY KEPT FOR DEMONSTRATION OF PROJECT EVOLUTION.**

In [12]:
# class TagVocabulary:
#   #initialization function
#   def __init__(self):
#     self.tags = [
#             # Content Creation
#             "Content Writing", "Copywriting", "Blog Writing", "Article Writing",
#             "Creative Writing", "Technical Writing", "Content Strategy",

#             # Marketing
#             "Social Media Marketing", "Digital Marketing", "Email Marketing",
#             "Marketing Strategy", "Brand Marketing", "Influencer Marketing",

#             # Social Media
#             "Social Media", "Facebook Marketing", "Instagram Marketing",
#             "Twitter Marketing", "LinkedIn Marketing", "TikTok Marketing",

#             # Analytics & Testing
#             "A/B Testing", "Analytics", "Performance Tracking", "Data Analysis",
#             "Audience Research", "Market Research",

#             # Advertising
#             "Online Advertising", "Social Media Ads", "Google Ads",
#             "Facebook Ads", "Digital Advertising",

#             # Skills & Techniques
#             "Communication Skills", "Writing Skills", "Creative Skills",
#             "Marketing Skills", "Design Skills",

#             # Strategy & Planning
#             "Content Planning", "Marketing Planning", "Campaign Strategy",
#             "Audience Targeting", "Customer Engagement"
#         ]

#     print(f"Tag vocabulary initialized with {len(self.tags)} tags")

#     #this will return list of tags in our vocabulary
#     def get_tags(self) -> List[str]:
#         return self.tags.copy()

#     #this will add a new tag in our vocabulary
#     def add_tag(self, new_tag: str):
#         if new_tag not in self.tags:
#             self.tags.append(new_tag)
#             print(f"Added new tag: {new_tag}")
#         else:
#             print(f"Tag '{new_tag}' already exists")

In [13]:
# class BasicTagger:
#   #initializer function
#     def __init__(self):
#       self.encoder = TextEncoder()
#       self.vocabulary = TagVocabulary()

#       self.tag_embeddings = self._encode_all_tags()

#     #function to encode all tags before hand
#     def _encode_all_tags(self) -> Dict[str, np.ndarray]:
#         tag_embeddings = {}
#         for tag in self.vocabulary.tags:
#             embedding = self.encoder.encode_text(tag)
#             tag_embeddings[tag] = embedding

#         return tag_embeddings

#     #finds tags from our vocaublary that are applicable according to our text input
#     def find_matching_tags(self, input_text: str, top_k: int = 10) -> List[Tuple[str, float]]:
#         # Encode the input text
#         input_embedding = self.encoder.encode_text(input_text)

#         similarities = []

#         for tag_name, tag_embedding in self.tag_embeddings.items():
#             # Calculate cosine similarity
#             similarity = cosine_similarity(
#                 input_embedding.reshape(1, -1),
#                 tag_embedding.reshape(1, -1)
#             )[0][0]

#             similarities.append((tag_name, float(similarity)))

#         similarities.sort(key=lambda x: x[1], reverse=True)

#         return similarities[:top_k]

In [14]:
# # Test our basic tagger
# def test_basic_tagger():
#     tagger = BasicTagger()

#     test_text = """
#     Creating social media posts is a great way to hone your content writing skills.
#     Since posts are typically very short, snappy, and quick, you can easily try out
#     different styles of writing and see what people respond to. It's easy to change
#     direction and adapt if you need to tweak your writing style since social media
#     posts are typically fluid and changeable by nature. You can also practice A/B
#     testing with your social media ads—try writing two different posts and sending
#     it to similar demographics and see which one performs better.
#     """

#     print("Input text:")
#     print(test_text)

#     # Find matching tags
#     matching_tags = tagger.find_matching_tags(test_text, top_k=15)

#     print("Top matching tags:")
#     for i, (tag, score) in enumerate(matching_tags, 1):
#         print(f"{i:2d}. {tag:<25} (Score: {score:.3f})")

In [15]:
# if __name__ == "__main__":
#   test_basic_tagger()

### **Dynamic Tag Generation**

### **This is the approach that is being followed right now, it is able to fix the problems and limitation that we were facing in the above approach**

In [16]:
#this extracts key phrases from text using linguistic patterns and statistical methods.
class KeyPhraseExtractor:
    def __init__(self):
        # trying to use spacy model for linguistic analysis
        try:
            self.nlp = spacy.load("en_core_web_sm")
        except:
            print("Please install spacy model: python -m spacy download en_core_web_sm")
            raise

        #I have used Parts of Speech(POS) concept for tag extraction
        self.phrase_patterns = [
            #1-word phrases
            ["NOUN"],
            ["PROPN"],

            # 2-word phrases
            ["ADJ", "NOUN"],
            ["ADJ", "PROPN"],
            ["NOUN", "NOUN"],
            ["PROPN", "NOUN"],
            ["NOUN", "PROPN"],

            # 3-word phrases
            ["ADJ", "NOUN", "NOUN"],
            ["NOUN", "NOUN", "NOUN"],
            ["PROPN", "PROPN", "PROPN"],
            ["ADJ", "ADJ", "NOUN"],
            ["NOUN", "VERB", "NOUN"],

            # 4-word phrases
            ["ADJ", "NOUN", "NOUN", "NOUN"],
            ["NOUN", "NOUN", "NOUN", "NOUN"],
            ["ADJ", "ADJ", "NOUN", "NOUN"],
            # ["NOUN", "NOUN", "VERB", "NOUN"],
            # ["ADJ", "NOUN", "VERB", "NOUN"],
            # ["NOUN", "NOUN", "NOUN", "VERB"],

            # Verb forms (gerunds)
            # ["VERB"],  # Will filter for -ing forms
            # ["ADJ", "VERB"],
        ]

        #common compound terms that should stay together
        self.compound_terms = {
            "machine learning", "deep learning", "natural language processing",
            "neural network", "data science", "artificial intelligence",
            "computer vision", "big data", "decision making",
            "supply chain", "customer relationship", "human resources",
            "business process"
        }


    #applying Named Entity Recognition(NER) for extracting entities like ORG, PERSON, GPE, DATE, etc.
    def extract_named_entities(self, text: str) -> List[Tuple[str, str, int]]:
      #finalised using spacy NER, i tried BERT based NER as well but it was also giving near to similar results
      #hence why to waste time and space and make our model more complex to understand
      doc = self.nlp(text)

      #count frequency of each entity
      entity_counter = Counter()
      entity_labels = {}


      for ent in doc.ents:
          entity_norm = ent.text.lower().strip()
          if ent.label_ == "CARDINAL":
              continue

          #dont use very short entities. (i did this because after some testing on phrases NER was catching some garbage as well)
          if len(ent.text) < 3 and ent.label_ not in ["ORG", "GPE", "PERSON"]:
              continue

          entity_counter[entity_norm] += 1
          #storing original label
          if entity_norm not in entity_labels:
              entity_labels[entity_norm] = (ent.text, ent.label_)

      #putting the output in proper format
      entities = []
      for norm_text, freq in entity_counter.items():
          original_text, label = entity_labels[norm_text]
          entities.append((original_text, label, freq))

      return entities


    #using noun_chunks from spacy
    def extract_noun_chunks(self, text: str) -> List[Tuple[str, int]]:
        doc = self.nlp(text)

        chunk_counter = Counter()

        # Words that shouldn't start/end chunks
        bad_starts = {'which', 'that', 'this', 'these', 'those', 'your', 'my', 'our', 'their', 'a', 'an', 'the', "it's", 'its'}
        bad_ends = {'one', 'ones', 'thing', 'things', 'way', 'ways'}

        for chunk in doc.noun_chunks:
            chunk_text = chunk.text.strip()
            chunk_words = chunk_text.lower().split()

            # Skip if starts/ends badly
            if chunk_words and (chunk_words[0] in bad_starts or chunk_words[-1] in bad_ends):
                continue

            #taking only multiword phrases because one word phrase will already get caught in POS
            if len(chunk_words) >= 2:
                chunk_lower = chunk_text.lower()
                chunk_counter[chunk_lower] += 1

        return list(chunk_counter.items())


    #updating extract phrases function to combine POS patterns from before and noun chunks and NER entitites as well
    def extract_phrases_with_metadata(self, text: str, min_freq: int = 1) -> List[Tuple[str, int, dict]]:
      doc = self.nlp(text.lower())

      phrase_data = {} #this will contain freqency, source, entity type and original twxt

      #extraction from ner
      ner_entities = self.extract_named_entities(text)
      for entity_text, entity_label, freq in ner_entities:
          phrase_lower = entity_text.lower()

          if phrase_lower not in phrase_data:
              phrase_data[phrase_lower] = {
                  'freq': freq,
                  'source': 'ner',
                  'entity_type': entity_label,
                  'original_text': entity_text
              }
          else:
              if freq > phrase_data[phrase_lower]['freq']:
                  phrase_data[phrase_lower]['freq'] = freq
                  phrase_data[phrase_lower]['entity_type'] = entity_label

      #extraction fron noun_chunks
      noun_chunks = self.extract_noun_chunks(text)
      for chunk_text, freq in noun_chunks:
          if chunk_text not in phrase_data:
              phrase_data[chunk_text] = {
                  'freq': freq,
                  'source': 'noun_chunk',
                  'entity_type': None,
                  'original_text': chunk_text
              }

      #small cbeck in compund terms(trial)
      text_lower = text.lower()
      for compound in self.compound_terms:
          count = text_lower.count(compound)
          if count > 0:
              if compound not in phrase_data:
                  phrase_data[compound] = {
                      'freq': count,
                      'source': 'pos',
                      'entity_type': None,
                      'original_text': compound
                  }
              else:
                  phrase_data[compound]['freq'] = max(phrase_data[compound]['freq'], count)

      #POS matching algorithm
      for sentence in doc.sents:
          tokens = list(sentence)
          for start_idx in range(len(tokens)):
              for pattern in self.phrase_patterns:
                  end_idx = start_idx + len(pattern)
                  if end_idx <= len(tokens):
                      span_tokens = tokens[start_idx:end_idx]
                      pos_sequence = [token.pos_ for token in span_tokens]

                      if pos_sequence == pattern:
                          phrase_tokens = []
                          valid = True

                          for i, token in enumerate(span_tokens):
                              if len(pattern) == 1:
                                  if token.is_stop:
                                      valid = False
                                      break

                                  meaningless_words = {
                                      'people', 'person', 'thing', 'things', 'way', 'ways',
                                      'time', 'times', 'place', 'places', 'nature', 'direction',
                                      'fact', 'case', 'point', 'example', 'kind', 'type',
                                      'one', 'two', 'three', 'four', 'five'
                                  }

                                  if token.text.lower() in meaningless_words:
                                      valid = False
                                      break

                                  if token.pos_ == "VERB":
                                      if not token.text.endswith("ing"):
                                          valid = False
                                          break

                                  if len(token.text) < 3:
                                      valid = False
                                      break

                              phrase_tokens.append(token.text)

                          if valid and phrase_tokens:
                              phrase = " ".join(phrase_tokens)
                              phrase = re.sub(r'\s+', ' ', phrase).strip()

                              is_subset = False
                              for existing in phrase_data:
                                  if phrase in existing and phrase != existing:
                                      is_subset = True
                                      break

                              if not is_subset and phrase:
                                  if phrase not in phrase_data:
                                      phrase_data[phrase] = {
                                          'freq': 1,
                                          'source': 'pos',
                                          'entity_type': None,
                                          'original_text': phrase
                                      }
                                  else:
                                      phrase_data[phrase]['freq'] += 1
      #clesning for output
      phrases_with_metadata = []
      for phrase, data in phrase_data.items():
          if data['freq'] >= min_freq:
              metadata = {
                  'source': data['source'],
                  'entity_type': data['entity_type']
              }
              display_text = data.get('original_text', phrase)
              phrases_with_metadata.append((display_text, data['freq'], metadata))

      phrases_with_metadata.sort(key=lambda x: (x[1], len(x[0].split())), reverse=True)

      return phrases_with_metadata


    def extract_phrases(self, text: str, min_freq: int = 1) -> List[Tuple[str, int]]:
        phrases_with_metadata = self.extract_phrases_with_metadata(text, min_freq)
        return [(phrase, freq) for phrase, freq, metadata in phrases_with_metadata]

In [17]:
#this is used to score adn filter the phrases that we extracted above to identify best tags in our input sentance
class PhraseScorer:
    def __init__(self, encoder=None):

        self.encoder = encoder

        # Common/generic words that make poor tags
        self.generic_words = {
            'way', 'ways', 'thing', 'things', 'people', 'person', 'time', 'times',
            'place', 'places', 'day', 'days', 'year', 'years', 'good', 'bad',
            'great', 'nice', 'sure', 'certain', 'different', 'same', 'other',
            'new', 'old', 'high', 'low', 'large', 'small', 'long', 'short',
            'easy', 'hard', 'simple', 'complex', 'nature', 'type', 'types',
            'kind', 'kinds', 'lot', 'lots', 'direction', 'need', 'needs'
        }



    #encorporated NER entities to have higher weightage
    def calculate_phrase_scores_with_ner(self, phrases_with_metadata: List[Tuple[str, int, dict]],
                                    text_length: int) -> List[Tuple[str, float, dict]]:

      scored_phrases = []

      #get max frequency for normalization
      max_freq = 1
      if phrases_with_metadata:
          frequencies = []
          for phrase, freq, metadata in phrases_with_metadata:
              frequencies.append(freq)
          max_freq = max(frequencies)

      for phrase, freq, metadata in phrases_with_metadata:
          #initialize scores
          scores = {
              'frequency': 0.0,
              'specificity': 0.0,
              'length': 0.0,
              'completeness': 0.0,
              'entity_bonus': 0.0
          }

          #frequency score
          scores['frequency'] = min(freq / max_freq, 1.0) * 0.3

          #secificity score
          words = phrase.lower().split()
          generic_count = sum(1 for word in words if word in self.generic_words)
          scores['specificity'] = (1 - generic_count / len(words)) * 0.25

          #length score
          if len(words) == 1:
              scores['length'] = 0.65
          elif len(words) == 2:
              scores['length'] = 0.85
          elif len(words) == 3:
              scores['length'] = 0.90
          elif len(words) == 4:
              scores['length'] = 1.0
          else:
              scores['length'] = 0.4
          scores['length'] *= 0.15

          # completeness score
          incomplete_markers = {'of', 'to', 'for', 'with', 'and', 'or', 'but', 'the', 'a', 'an'}
          is_complete = (words[0] not in incomplete_markers and
                        words[-1] not in incomplete_markers)
          scores['completeness'] = 1.0 if is_complete else 0.5
          scores['completeness'] *= 0.05

          #-----------------------------------------------------------------
          #NER scores
          if metadata.get('source') == 'ner' and metadata.get('entity_type'):
            entity_type = metadata['entity_type']
            if entity_type == 'PERSON':
                scores['entity_bonus'] = 0.05  # Was 0.20
            elif entity_type == 'ORG':
                scores['entity_bonus'] = 0.05  # Was 0.20
            elif entity_type == 'GPE':
                scores['entity_bonus'] = 0.05  # Was 0.25
            elif entity_type == 'DATE':
                scores['entity_bonus'] = 0.03  # Was 0.15
            else:
                scores['entity_bonus'] = 0.02  # Was 0.10

          #calculate total score
          total_score = sum(scores.values())

          scored_phrases.append((phrase, total_score, metadata))

      scored_phrases.sort(key=lambda x: x[1], reverse=True)

      return scored_phrases



    def semantic_clustering(self, phrases_with_scores: List[Tuple[str, float]],
                          encoder, similarity_threshold: float = 0.85) -> List[Tuple[str, float]]:

        if not phrases_with_scores:
            return []

        #extract phrases and scores
        phrases = [phrase for phrase, _ in phrases_with_scores]
        scores = [score for _, score in phrases_with_scores]

        #generate embeddings for all phrases

        embeddings = []
        for phrase in phrases:
            embedding = encoder.encode_text(phrase)
            embeddings.append(embedding)

        embeddings = np.array(embeddings)

        #calculate pairwise similarities
        similarities = cosine_similarity(embeddings)

        #check for substring relationships and boost their similarity
        for i in range(len(phrases)):
          for j in range(i + 1, len(phrases)):
              phrase_i_lower = phrases[i].lower()
              phrase_j_lower = phrases[j].lower()

              #if one phrase contains the other, boost similarity
              if phrase_i_lower in phrase_j_lower or phrase_j_lower in phrase_i_lower:
                  #set similarity to at least 90% for substring matches
                  similarities[i][j] = max(similarities[i][j], 0.9)
                  similarities[j][i] = similarities[i][j]


                  # print(f"DEBUG: Boosted similarity between '{phrases[i]}' and '{phrases[j]}' to {similarities[i][j]:.3f}")


        # Simple clustering: for each phrase, find all phrases similar to it
        clustered = set()  # Keep track of phrases already assigned to a cluster
        clusters = []

        for i in range(len(phrases)):
            if i in clustered:
                continue

            #start a new cluster with phrase i
            cluster = [(i, phrases[i], scores[i])]
            clustered.add(i)

            #find all phrases similar to phrase i
            for j in range(i + 1, len(phrases)):
                if j in clustered:
                    continue

                if similarities[i][j] >= similarity_threshold:
                    cluster.append((j, phrases[j], scores[j]))
                    clustered.add(j)

            clusters.append(cluster)

        #select best phrase from each cluster
        best_phrases = []

        for cluster in clusters:
            #sort by score (descending) and phrase length (descending for tie-breaking)
            cluster.sort(key=lambda x: (x[2], len(x[1])), reverse=True)

            #add the best phrase from this cluster
            _, best_phrase, best_score = cluster[0]
            best_phrases.append((best_phrase, best_score))

            #debuging--
            # if len(cluster) > 1:
            #     merged = [item[1] for item in cluster[1:]]
            #     print(f"Merged into '{best_phrase}': {merged}")


        best_phrases.sort(key=lambda x: x[1], reverse=True)

        return best_phrases

    #This removes similar or redundant phrases keeping the ones that are most meaningful
    def filter_similar_phrases_with_metadata(self, scored_phrases: List[Tuple[str, float, dict]],
                                        similarity_threshold: float = 0.85) -> List[Tuple[str, float, dict]]:
          if not scored_phrases:
              return []

          filtered = []

          embedding_cache = {}

          for phrase, score, metadata in scored_phrases:
              phrase_lower = phrase.lower()
              words = set(phrase_lower.split())

              should_keep = True
              phrases_to_remove = []

              #calculate embedding for current phrase if we have encoder
              if self.encoder and phrase_lower not in embedding_cache:
                  embedding_cache[phrase_lower] = self.encoder.encode_text(phrase)

              for i, (kept_phrase, kept_score, kept_metadata) in enumerate(filtered):
                  kept_lower = kept_phrase.lower()
                  kept_words = set(kept_lower.split())

                  #skip if exact same phrase
                  if phrase_lower == kept_lower:
                      should_keep = False
                      break

                  #check semantic similarity if encoder available
                  if self.encoder:
                      #get or calculate embedding for kept phrase
                      if kept_lower not in embedding_cache:
                          embedding_cache[kept_lower] = self.encoder.encode_text(kept_phrase)

                      #calculate similarity
                      semantic_similarity = cosine_similarity(
                          embedding_cache[phrase_lower].reshape(1, -1),
                          embedding_cache[kept_lower].reshape(1, -1)
                      )[0][0]

                      if semantic_similarity > similarity_threshold:
                          #prefer NER entities
                          current_is_ner = metadata.get('source') == 'ner'
                          kept_is_ner = kept_metadata.get('source') == 'ner'

                          if current_is_ner and not kept_is_ner:
                              phrases_to_remove.append(i)
                          elif kept_is_ner and not current_is_ner:
                              should_keep = False
                              break
                          else:
                              #same source type - prefer shorter/cleaner or higher score
                              if len(words) < len(kept_words) and score > kept_score * 0.9:
                                  phrases_to_remove.append(i)
                              elif score > kept_score:
                                  phrases_to_remove.append(i)
                              else:
                                  should_keep = False
                                  break

                  #original subset logic
                  elif words.issubset(kept_words) or kept_words.issubset(words):
                      len_diff = abs(len(words) - len(kept_words))
                      score_diff = abs(score - kept_score)

                      if score_diff < 0.15 and len_diff >= 1:
                          current_is_ner = metadata.get('source') == 'ner'
                          kept_is_ner = kept_metadata.get('source') == 'ner'

                          if current_is_ner and not kept_is_ner:
                              phrases_to_remove.append(i)
                          elif kept_is_ner and not current_is_ner:
                              should_keep = False
                              break
                          else:
                              if len(words) > len(kept_words):
                                  phrases_to_remove.append(i)
                              else:
                                  should_keep = False
                                  break
                      else:
                          if score > kept_score:
                              phrases_to_remove.append(i)
                          else:
                              should_keep = False
                              break

              if phrases_to_remove:
                  for idx in reversed(phrases_to_remove):
                      filtered.pop(idx)

              if should_keep:
                  filtered.append((phrase, score, metadata))

          return filtered

    #calculate score without ner
    def calculate_phrase_scores(self, phrases: List[Tuple[str, int]],
                                text_length: int) -> List[Tuple[str, float]]:
        scored_phrases = []

        #get max frequency for normalization
        max_freq = max([freq for _, freq in phrases]) if phrases else 1

        for phrase, freq in phrases:
            #initialize scores
            scores = {
                'frequency': 0.0,
                'specificity': 0.0,
                'length': 0.0,
                'completeness': 0.0
            }

            #frequency score
            scores['frequency'] = min(freq / max_freq, 1.0) * 0.3

            #specificity score
            words = phrase.lower().split()
            generic_count = sum(1 for word in words if word in self.generic_words)
            scores['specificity'] = (1 - generic_count / len(words)) * 0.25

            # 3.length score
            if len(words) == 1:
                scores['length'] = 0.65
            elif len(words) == 2:
                scores['length'] = 0.85
            elif len(words) == 3:
                scores['length'] = 0.90
            elif len(words) == 4:
                scores['length'] = 1.0
            else:
                scores['length'] = 0.4
            scores['length'] *= 0.15

            #trial
            incomplete_markers = {'of', 'to', 'for', 'with', 'and', 'or', 'but', 'the', 'a', 'an'}
            is_complete = (words[0] not in incomplete_markers and
                          words[-1] not in incomplete_markers)
            scores['completeness'] = 1.0 if is_complete else 0.5
            scores['completeness'] *= 0.1

            #calculate score
            total_score = sum(scores.values())

            scored_phrases.append((phrase, total_score))

        scored_phrases.sort(key=lambda x: x[1], reverse=True)

        return scored_phrases


    def filter_similar_phrases(self, scored_phrases: List[Tuple[str, float]]) -> List[Tuple[str, float]]:
        if not scored_phrases:
            return []

        filtered = []

        for phrase, score in scored_phrases:
            phrase_lower = phrase.lower()
            words = set(phrase_lower.split())

            should_keep = True
            phrases_to_remove = []

            for i, (kept_phrase, kept_score) in enumerate(filtered):
                kept_lower = kept_phrase.lower()
                kept_words = set(kept_lower.split())

                #skip if exact same phrase is there
                if phrase_lower == kept_lower:
                    should_keep = False
                    break

                #handle subset relationships
                if words.issubset(kept_words) or kept_words.issubset(words):
                    len_diff = abs(len(words) - len(kept_words))
                    score_diff = abs(score - kept_score)

                    if score_diff < 0.15 and len_diff >= 1:
                        if len(words) > len(kept_words):
                            phrases_to_remove.append(i)
                        else:
                            should_keep = False
                            break
                    else:
                        if score > kept_score:
                            phrases_to_remove.append(i)
                        else:
                            should_keep = False
                            break

            if phrases_to_remove:
                for idx in reversed(phrases_to_remove):
                    filtered.pop(idx)

            if should_keep:
                filtered.append((phrase, score))

        return filtered

In [18]:
#Complete dynamic tagging class
class DynamicTagger:
    def __init__(self, encoder_model="sentence-transformers/all-MiniLM-L6-v2"):
        self.encoder = TextEncoder(encoder_model)
        self.extractor = KeyPhraseExtractor()
        self.scorer = PhraseScorer(self.encoder)
        print("DynamicTagger ready!", flush=True)

    def generate_tags(self, text: str, max_tags: int = 10, min_score: float = 0.6,
                     use_semantic_clustering: bool = True,
                     similarity_threshold: float = 0.70) -> List[Tuple[str, float]]:

        phrases = self.extractor.extract_phrases(text)

        if not phrases:
            return []

        word_count = len(text.split())
        scored_phrases = self.scorer.calculate_phrase_scores(phrases, word_count)

        if use_semantic_clustering:
            #use semantic clustering to group similar phrases
            filtered_phrases = self.scorer.semantic_clustering(
                scored_phrases,
                self.encoder,
                similarity_threshold
            )
        else:
            filtered_phrases = self.scorer.filter_similar_phrases(scored_phrases)

        #apply semantic relevance using embeddings
        text_embedding = self.encoder.encode_text(text)

        #combine quality score with semantic relevance
        final_scores = []
        for phrase, quality_score in filtered_phrases:
            #get semantic similarity between phrase and full text
            phrase_embedding = self.encoder.encode_text(phrase)

            #calculate cosine similarity
            semantic_score = cosine_similarity(
                text_embedding.reshape(1, -1),
                phrase_embedding.reshape(1, -1)
            )[0][0]

            #combine scores (70% quality, 30% semantic)
            combined_score = (quality_score * 0.7) + (semantic_score * 0.3)

            final_scores.append((phrase, combined_score))

        final_scores.sort(key=lambda x: x[1], reverse=True)

        quality_tags = [(tag, score) for tag, score in final_scores if score > min_score]

        if len(quality_tags) < 5 and len(final_scores) >= 5:
            quality_tags = final_scores[:5]

        return quality_tags[:max_tags]

    #this returns only tags
    def tag_text(self, text: str, max_tags: int = 7) -> List[str]:
        tag_scores = self.generate_tags(text, max_tags)
        return [tag for tag, _ in tag_scores]

    #this returns tags with scores
    def tag_text_with_scores(self, text: str, max_tags: int = 7) -> List[Tuple[str, float]]:
        return self.generate_tags(text, max_tags)

    #tags, score, source
    def generate_tags_with_source(self, text: str, max_tags: int = 10, min_score: float = 0.3,
                            use_semantic_clustering: bool = True,
                            similarity_threshold: float = 0.75) -> List[Tuple[str, float, str]]:
        #extract phrases this time along with NER
        phrases_with_metadata = self.extractor.extract_phrases_with_metadata(text)

        if not phrases_with_metadata:
            return []

        #score including ner
        word_count = len(text.split())
        scored_phrases = self.scorer.calculate_phrase_scores_with_ner(phrases_with_metadata, word_count)


        if use_semantic_clustering:
          # First extract just phrases and scores for clustering
          phrases_scores = [(phrase, score) for phrase, score, _ in scored_phrases]

          # Apply semantic clustering
          clustered = self.scorer.semantic_clustering(
              phrases_scores,
              self.encoder,
              similarity_threshold
          )

          # Rebuild with metadata - find original metadata for each clustered phrase
          filtered_phrases = []
          for phrase, score in clustered:
              # Find the original metadata
              for orig_phrase, orig_score, metadata in scored_phrases:
                  if orig_phrase == phrase:
                      filtered_phrases.append((phrase, score, metadata))
                      break
        else:
             filtered_phrases = self.scorer.filter_similar_phrases_with_metadata(scored_phrases)

        #applying semantic embeddings for clustering similar phrases
        text_embedding = self.encoder.encode_text(text)

        #combine scores
        final_scores = []
        for phrase, quality_score, metadata in filtered_phrases:
            #get semantic similairty of code and full text
            phrase_embedding = self.encoder.encode_text(phrase)

            #calculate cosine similarity
            semantic_score = cosine_similarity(
                text_embedding.reshape(1, -1),
                phrase_embedding.reshape(1, -1)
            )[0][0]

            #combine scores
            combined_score = (quality_score * 0.85) + (semantic_score * 0.15)

            #keep source
            source = metadata.get('source', 'pos')
            final_scores.append((phrase, combined_score, source))


        final_scores.sort(key=lambda x: x[1], reverse=True)

        #normalize scores
        if final_scores:
            max_score = final_scores[0][1]
            if max_score > 0:
                final_scores = [(tag, score / max_score, source) for tag, score, source in final_scores]

        quality_tags = [(tag, score, source) for tag, score, source in final_scores if score > min_score]

        if len(quality_tags) < 5 and len(final_scores) >= 5:
            quality_tags = final_scores[:5]

        return quality_tags[:max_tags]

# **Testing**